In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [10]:
df_sticker_send = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/BBM-STICKER-SEND/")
df_sticker_receiv = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/BBM-STICKER-RECEIVED/")
df_sticker_down = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/BBM-STICKER-DOWNLOAD/")
df_sticker_click = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/BBM-STICKER-SEND/")
df_sticker_sponsor_click= sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/BBM-STICKER-SEND/")


In [11]:
df_sticker_send = df_sticker_send.withColumn("dataset_name",lit("STICKER_SEND"))
df_sticker_receiv = df_sticker_receiv.withColumn("dataset_name",lit("STICKER_RECEIVE"))
df_sticker_down = df_sticker_down.withColumn("dataset_name",lit("STICKER_DOWNLOAD"))
df_sticker_click = df_sticker_click.withColumn("dataset_name",lit("STICKER_CLICK"))
df_sticker_sponsor_click = df_sticker_sponsor_click.withColumn("dataset_name",lit("STICKER_SPONSOR_CLICK"))

In [12]:
first_concat = df_sticker_send.union(df_sticker_receiv)
second_concat = first_concat.union(df_sticker_down)
third_concat = second_concat.union(df_sticker_click)
fourth_concat = third_concat.union(df_sticker_sponsor_click)


In [13]:
fourth_concat.columns

['id', 'visit_id', 'user_id', 'name', 'properties', 'time', 'dataset_name']

In [ ]:
fourth_concat.write.mode('overwrite').parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/concatenated/")

In [17]:
final_ds = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/concatenated/")

In [18]:
#function to split a single column to multiple columns
def extract_json(line, schema_names):
    result = [x for i,x in enumerate(line) if i!=4]
    properties = json.loads(line[4])
    flattened = []     
    for key in schema_names:
        if key in properties:
            if type(properties[key]) == unicode :
                flattened.append(properties[key].encode('utf-8'))
            else:
                flattened.append(str(properties[key]))
                
        else:
            flattened.append("NULL")
    result = list(result) + flattened
    return result

In [19]:
json_schema_final = spark.read.json(final_ds.rdd.map(lambda row: row.properties)).schema

In [34]:

# #download
old_column_names_send = df_sticker_down.drop('properties').columns
new_items_send = ['user_id_n' if x == 'user_id' else 'app_version_n' if  x == 'app_version' else 'city_n' if  x == 'city' else 'connection_n' if  x == 'connection' else 'country_n' if  x == 'country' else 'device_brand_n' if  x == 'device_brand' else 'device_model_n' if  x == 'device_model' else 'has_nfc_n' if  x == 'has_nfc' else 'height_resolution_n' if  x == 'height_resolution' else 'new_user_n' if  x == 'new_user' else 'new_user_n_1' if  x == 'new_User' else 'os_version_n' if  x == 'os_version' else 'width_resolution_n' if  x == 'width_resolution' else x for x in json_schema_final.names]
fin_col_names_send = old_column_names_send + new_items_send



In [35]:
import json
df = final_ds.rdd.map(lambda x: extract_json(x, json_schema_final.names)).toDF(fin_col_names_send)

In [36]:
#deriving new columns
df = df.drop('user_id')
df = df.withColumn('City',when(df.City == 'NULL' ,df.city_n).otherwise(df.City))
df = df.drop('city_n')
df = df.withColumn('App_Version',when(df.App_Version == 'NULL' ,df.app_version_n).otherwise(df.App_Version))
df = df.drop('app_version_n')
df = df.withColumn('Connection',when(df.Connection == 'NULL' ,df.connection_n).otherwise(df.Connection))
df = df.drop('connection_n')
df = df.withColumn('Country',when(df.Country == 'NULL' ,df.country_n).otherwise(df.Country))
df = df.drop('country_n')
df = df.withColumn('Device_Brand',when(df.Device_Brand == 'NULL' ,df.device_brand_n).otherwise(df.Device_Brand))
df = df.drop('device_brand_n')
df = df.withColumn('Device_Model',when(df.Device_Model == 'NULL' ,df.device_model_n).otherwise(df.Device_Model))
df = df.drop('device_model_n')
df = df.withColumn('Has_NFC',when(df.Has_NFC == 'NULL' ,df.has_nfc_n).otherwise(df.Has_NFC))
df = df.drop('has_nfc_n')
df = df.withColumn('Height_Resolution',when(df.Height_Resolution == 'NULL' ,df.height_resolution_n).otherwise(df.Height_Resolution))
df = df.drop('height_resolution_n')
df = df.withColumn('New_User',when(df.New_User == 'NULL' ,df.new_user_n).otherwise(df.New_User))
df = df.withColumn('New_User',when(df.New_User == 'NULL' ,df.new_user_n_1).otherwise(df.New_User))
df = df.drop('new_user_n_1')
df = df.drop('new_user_n')
df = df.withColumn('OS_Version',when(df.OS_Version == 'NULL' ,df.os_version_n).otherwise(df.OS_Version))
df = df.drop('os_version_n')
df = df.withColumn('Width_Resolution',when(df.Width_Resolution == 'NULL' ,df.width_resolution_n).otherwise(df.Width_Resolution))
df = df.drop('width_resolution_n')

In [40]:
df.select('App_Version',
 'City',
 'Connection',
 'Country',
 'Device_Brand',
 'Device_Model',
 'Has_NFC',
 'Height_Resolution',
 'New_User',
 'OS_Version',
 'Width_Resolution',
 'action',
 'app_name',
 'c_pin',
 'carrier',
 'dataset_name',
 'eco_id',
 'event_action',
 'id',
 'lat',
 'license_type',
 'long',
 'name',
 'platform',
 'source',
 'sticker_id',
 'sticker_pack_id',
 'test_environment',
 'time',
 'u_pin',
 'user_id_n',
 'visit_id').write.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/transposed/final_ds/")

In [1]:
#reading in data again
df_new = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/transposed/final_ds/")

In [6]:
df_demo = sqlContext.read.parquet("gs://ds-user-demographic/parquet/tastes_mau_1024.parquet")

In [3]:
df_new = df_new.withColumn('date', concat(df_new.time.substr(1, 10)))
df_new = df_new.withColumn('month', concat(df_new.time.substr(6, 2)))

In [6]:
# print "Data schema :",len(df_new.columns),"\n"

# print "Total records in the data :",df_new.count(),"\n"





In [4]:
df_new.select('date').distinct().count()

45

In [5]:
df_new.select('month').distinct().count()

3

In [6]:
df_new.groupby('month').count().show()

+-----+----------+
|month|     count|
+-----+----------+
|   11| 186133605|
|   09| 469021062|
|   10|1676935490|
+-----+----------+



In [3]:
df_new.groupby('license_type').count().show()

+------------+----------+
|license_type|     count|
+------------+----------+
|discontinued|    517629|
|        free| 144115856|
|        paid|    549526|
|subscription|      2958|
|        NULL|2186904188|
+------------+----------+



In [ ]:
df_new.select('license_type','user_id_n').groupby('license_type').agg(countDistinct('user_id_n').alias('distinct users')).sort(col('distinct users').desc()).show()

In [12]:
df_new.select('user_id_n').distinct().count()

10535567

In [13]:
df_new.select('sticker_pack_id').distinct().count()

1733

In [14]:
df_new.select('sticker_id').distinct().count()

19981

In [ ]:
df_new.select('user_id_n').distinct().count()

In [19]:
data = df_new.groupby('dataset_name').agg(func.count("*").alias('cnt'))
total = data.select("cnt").agg({"cnt": "sum"}).collect().pop()['sum(cnt)']
data = data.withColumn("percentage", (data['cnt']/total) * 100)
data.sort(col('percentage').desc()).show()

+--------------------+---------+------------------+
|        dataset_name|      cnt|        percentage|
+--------------------+---------+------------------+
|        STICKER_SEND|603101791|25.860998091764596|
|       STICKER_CLICK|603101791|25.860998091764596|
|STICKER_SPONSOR_C...|603101791|25.860998091764596|
|     STICKER_RECEIVE|510706032|21.899068973258395|
|    STICKER_DOWNLOAD| 12078752|0.5179367514478129|
+--------------------+---------+------------------+



In [22]:
df_new.select('sticker_pack_id','sticker_id','platform','City','Country','user_id_n').groupby('sticker_pack_id','sticker_id','platform','City','Country').agg(countDistinct('user_id_n').alias('distinct users')).sort(col('distinct users').desc()).show(100)
# .select("page","visitor").groupBy('page).agg('page, countDistinct('visitor))

+-----------------+-------------------+-----------+-------------+-------+--------------+
|  sticker_pack_id|         sticker_id|   platform|         City|Country|distinct users|
+-----------------+-------------------+-----------+-------------+-------+--------------+
|              bbm|           ThumbsUp|app-android| Asia/Jakarta|     ID|       1572051|
|              bbm|            BigGrin|app-android| Asia/Jakarta|     ID|       1284914|
|          bbm2017|          bbm201704|app-android| Asia/Jakarta|     ID|       1190332|
|          bbm2017|          bbm201713|app-android| Asia/Jakarta|     ID|       1173971|
|          bbm2017|          bbm201703|app-android| Asia/Jakarta|     ID|       1127180|
|bbmfamilyanimated|bbmfamilyanimated04|app-android| Asia/Jakarta|     ID|        958620|
|              bbm|               Kiss|app-android| Asia/Jakarta|     ID|        940909|
|bbmfamilyanimated|bbmfamilyanimated12|app-android| Asia/Jakarta|     ID|        936203|
|bbmfamilyanimated|bb

In [25]:
from pyspark.sql.functions import col

active_base  = df_new.alias('a').join(df_demo.alias('b'),col('b.regid') == col('a.user_id_n')).select(col('user_id_n'),col('sticker_pack_id'),col('license_type'))

In [26]:
active_base.write.parquet("gs://ds-url-catag/temp/")

In [27]:
join_data = sqlContext.read.parquet("gs://ds-url-catag/temp/")

In [29]:
join_data.select('user_id_n').distinct().count()

10211909

In [30]:
join_data.select('sticker_pack_id').distinct().count()

1679

In [32]:
join_data.groupby('license_type').count().show()

+------------+----------+
|license_type|     count|
+------------+----------+
|        free|       226|
|        NULL|1347772828|
+------------+----------+

